In [62]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint

# reading docfiles
import docx2txt

#nltk
import nltk

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import pyLDAvis.lda_model
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [63]:
word_exclusions = pd.read_excel("input/word_exclusions.xlsx")

word_exclusions = word_exclusions[word_exclusions["exclude"] == 1]

word_exclusions = word_exclusions.drop(columns=['exclude', 'exclude_short_list'], axis = 1)

In [64]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(word_exclusions["word"])

In [90]:
df = pd.read_excel("input/app_v1/app_v1_feedback.xlsx")

In [92]:
feedback = df[df["category_id"] == 9]

feedback = feedback.drop(columns=['transcript_id', 'person_id', 'segment_id', 'context', 'lesson', 'category_id'], 
             axis = 1)

In [93]:
feedback['text_processed'] = \
feedback['text'].map(lambda x: re.sub('[,.!?]', '', x))

In [94]:
feedback['text_processed'] = \
feedback['text_processed'].map(lambda x: x.lower())

In [95]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), 
                                             deacc = True))

In [96]:
data = feedback.text_processed.values.tolist()
data_words = list(sent_to_words(data))

In [97]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['inhaltlich Natur schon Studium bekannt finden gut jetzt noch einmal wiederholen auffrischen', 'machen immer noch einmal wieder so bewussen Lage versetzen klar deutlich kommunizieren auch noch einmal nachfragen richtig verstehen glauben gerade Vorbereitung Geburt jetzt noch einmal ganz gut auffrischen']


In [98]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,                        # minimum reqd occurences of a word 
                             stop_words=stop_words,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ber', 'berhaupt', 'chlich', 'daf', 'hrend', 'nnen', 'nnte', 'rde', 'rden', 'rdest', 'ren', 'tats', 'tte'] not in stop_words.
  warnings.warn(


In [99]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  22.340425531914892 %


In [109]:
if __name__ == '__main__':

    # Build LDA Model
    lda_model = LatentDirichletAllocation(n_components=2,               # Number of topics
                                          max_iter=10,               # Max learning iterations
                                          learning_method='batch',   
                                          random_state=100,          # Random state
                                          evaluate_every = -1       # compute perplexity every n iters, default: Don't
                                         )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(n_components=2, random_state=100)


In [110]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -127.18212049276376
Perplexity:  5.997228655232542
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 2,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [111]:
# Define Search Param
search_params = {'n_components': [2, 5, 10], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [2, 5, 10]})

In [112]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 2}
Best Log Likelihood Score:  -35.220153201401665
Model Perplexity:  5.997071939585868


In [113]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,dominant_topic
Doc0,0.830000,0.170000,0
Doc1,0.830000,0.170000,0
Doc2,0.290000,0.710000,1
Doc3,0.830000,0.170000,0
Doc4,0.830000,0.170000,0
Doc5,0.870000,0.130000,0
Doc6,0.810000,0.190000,0
Doc7,0.870000,0.130000,0
Doc8,0.900000,0.100000,0
Doc9,0.760000,0.240000,0


In [114]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,29
1,1,18


In [115]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  return fun(*args, **kwargs)


,finden,gen,gut,immer,lernen,personlich
Topic0,19.472843,0.508859,22.47431,5.482155,0.558103,3.133049
Topic1,0.527157,12.491141,0.52569,0.517845,7.441897,3.866951


In [107]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=50)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

C:\Users\vanandkuma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  return fun(*args, **kwargs)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5
Topic 0,gen,lernen,personlich,finden,gut,immer
Topic 1,gut,finden,immer,personlich,lernen,gen


In [108]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(best_lda_model, data_vectorized, vectorizer)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.235559  0.0       1        1  66.289145
0     -0.235559  0.0       2        1  33.710855, topic_info=         Term       Freq      Total Category  logprob  loglift
1         gen  12.000000  12.000000  Default   6.0000   6.0000
4      lernen   7.000000   7.000000  Default   5.0000   5.0000
2         gut  20.000000  20.000000  Default   4.0000   4.0000
0      finden  18.000000  18.000000  Default   3.0000   3.0000
5  personlich   6.000000   6.000000  Default   2.0000   2.0000
3       immer   5.000000   5.000000  Default   1.0000   1.0000
2         gut  20.493448  20.989122   Topic1  -0.8314   0.3872
0      finden  17.756522  18.253580   Topic1  -0.9748   0.3835
3       immer   4.998973   5.487220   Topic1  -2.2423   0.3180
5  personlich   2.843262   6.503323   Topic1  -2.8066  -0.4162
4      lernen   0.509066   7.525491   Topic1  -4.5267  -2.2823
1         gen   0.464022  12.241265   Topic1  -4.6194  -2.8615
1         gen  11.777243  12.241265   Topic2  -0.7092   1.0487
4      lernen   7.016424   7.525491   Topic2  -1.2271   1.0173
5  personlich   3.660060   6.503323   Topic2  -1.8779   0.5125
3       immer   0.488247   5.487220   Topic2  -3.8923  -1.3320
0      finden   0.497059  18.253580   Topic2  -3.8744  -2.5161
2         gut   0.495674  20.989122   Topic2  -3.8772  -2.6585, token_table=      Topic      Freq        Term
term                             
0         1  0.986108      finden
1         2  0.980291         gen
2         1  0.952875         gut
3         1  0.911208       immer
4         1  0.132882      lernen
4         2  0.930172      lernen
5         1  0.461303  personlich
5         2  0.615070  personlich, R=6, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [ ]:
best_lda_model

In [ ]:
my_dict = {'Topic_' + str(i): [token for token, score in best_lda_model.show_topic(i, topn=10)] for i in range(0, best_lda_model.num_topics)}

print(my_dict)

AttributeError: 'LatentDirichletAllocation' object has no attribute 'num_components'